<a href="https://colab.research.google.com/github/yoshighosh/OCR-Testing/blob/main/Final_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preliminary Downloads

Tesseract Download

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 10s (486 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5

PDF file downloads

In [ ]:
!pip install pdf2image
!apt-get install poppler-utils
!pip install python-poppler-qt5

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 3s (47.7 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 160819 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... error
ERROR: Command errored out with exit status 1: /usr/b

PPTX file downloads

In [ ]:
!pip install python-pptx

     |████████████████████████████████| 9.3MB 4.4MB/s 
     |████████████████████████████████| 153kB 41.1MB/s 
  Created wheel for python-pptx: filename=python_pptx-0.6.19-cp37-none-any.whl size=469953 sha256=876cc3955494fbb386c171b83194cae6bd10359e4739092b017f7d384bc0d153
  Stored in directory: /root/.cache/pip/wheels/94/ef/02/9357c6781fbe3fee0e5e04bad23904d096e39d420423519631
Successfully built python-pptx


DOCX file downloads

In [ ]:
!pip install python-docx

     |████████████████████████████████| 5.6MB 4.3MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-cp37-none-any.whl size=184508 sha256=fae7a29289763ecb50c274312e8ccbffa31e1e2b6dcbbcefe02bced6d863e027
  Stored in directory: /root/.cache/pip/wheels/a6/90/f1/a7cb70b38633ae04e7fb963b1c70f63fd6fc01c075b8230adc
Successfully built python-docx


#Import Required Libraries

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow 
import pytesseract
from pytesseract import Output
import nltk
from nltk import word_tokenize
nltk.download('punkt')
from nltk import ngrams, FreqDist
import pandas as pd
import matplotlib.pyplot as plt
nltk.download('stopwords') 
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from pdf2image import convert_from_path
from IPython.display import Image
from pptx import Presentation
import glob
import docx

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


#Setting up OCR for each file type

Grayscale Function (Image and PDF)

In [ ]:
def get_grayscale(image):
  return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

Image Files

In [ ]:
def getImageText(filename):
  img = cv2.imread(filename)
  grayscale = get_grayscale(img)
  custom_config = r'-l eng --psm 6'
  text = pytesseract.image_to_string(grayscale, config=custom_config)
  return text

PDF Files

In [ ]:
def getPDFText(filename):
  pages = convert_from_path(filename, 500)
  pagenumber = 1
  page_paths = []
  for page in pages:
    filename = "page"+ str(pagenumber) + ".jpg"
    pagenumber += 1
    page_paths.append(filename)
    page.save(filename, 'JPEG')
  images = []
  for page in page_paths:
    images.append(cv2.imread(page))
  for image in images:
    image = get_grayscale(image)
  custom_config = r'-l eng --psm 6'
  text = ""
  for grayscale in images:
    text += pytesseract.image_to_string(grayscale, config=custom_config)
  return text


PPTX Files

In [ ]:
def getPPTXText(filename):
  for eachfile in glob.glob(filename):
      prs = Presentation(eachfile)
      fullText = []
      for slide in prs.slides:
          for shape in slide.shapes:
              if hasattr(shape, "text"):
                  fullText.append(shape.text)
  return '\n'.join(fullText)

DOCX Text

In [ ]:
def getDOCXText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

# Detecting file type

In [ ]:
def detectFileType(filename):
  extension = filename[filename.rfind(".")+1:]
  return extension

def getFileName(filepath):
  filename = filepath[filepath.rfind("/")+1:]
  return filename

#read file binary, detect file


In [ ]:
filepaths = ["/content/drive/MyDrive/Aroshi_highSchool/SEAP vounteer/pptx files/Leaf Disk Lab Poster (1).pptx", '/content/drive/MyDrive/Aroshi_highSchool/SEAP vounteer/image files/New Image.jpg', '/content/drive/MyDrive/Aroshi_highSchool/SEAP vounteer/image files/Picture_003.jpg', '/content/drive/MyDrive/Aroshi_highSchool/SEAP vounteer/image files/Picture_025.tif', '/content/drive/MyDrive/Aroshi_highSchool/SEAP vounteer/image files/Picture_029.tif', '/content/drive/MyDrive/Aroshi_highSchool/SEAP vounteer/pdf files/Depth-First Search and Linear Graph Algorithms - Tarjan (1972).pdf', '/content/drive/MyDrive/Aroshi_highSchool/SEAP vounteer/pdf files/Efficient String Matching - An Aid to Bibliographic Search - Aho-Corasick (1975).pdf', '/content/drive/MyDrive/Aroshi_highSchool/SEAP vounteer/docx files/SampleResume.docx']

text = {}

for filepath in filepaths:
  file_type = detectFileType(filepath)
  filename = getFileName(filepath)
  string_text = ""
  if file_type == "jpg":
    string_text = getImageText(filepath)
  elif file_type == "pdf":
    string_text = getPDFText(filepath)
  elif file_type == "pptx":
    string_text = getPPTXText(filepath)
  elif file_type == "docx":
    string_text = getDOCXText(filepath)
  text[filename] = string_text


In [ ]:
for file in text:
  print(file)

Leaf Disk Lab Poster (1).pptx
New Image.jpg
Picture_003.jpg
Picture_025.tif
Picture_029.tif
Depth-First Search and Linear Graph Algorithms - Tarjan (1972).pdf
Efficient String Matching - An Aid to Bibliographic Search - Aho-Corasick (1975).pdf
SampleResume.docx


#Preprocessing text

In [ ]:
def preprocessing(text):
  words = word_tokenize(text)
  updated_text = [word.lower() for word in words]
  final_text = [x for x in updated_text if x.isalpha()]
  stop_words = stopwords.words('english')
  final_text = [word for word in final_text if word not in stop_words]
  lemmatizer = WordNetLemmatizer()
  for word in final_text:
    word = lemmatizer.lemmatize(word)
  final_string = " ".join(final_text)
  return final_string

In [ ]:
query = input("Enter your query: ")
print(query)

Enter your query: Photosynthesis and substrate concentration
Photosynthesis and substrate concentration


In [ ]:
final_query = preprocessing(query)

In [ ]:
for file in text:
  text[file] = preprocessing(text[file])

text["query"] = final_query
#text.pop("query")

In [ ]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([text[file] for file in text])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
#print(dense)
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

display(df)

,ab,abdomen,abilities,able,absence,abstract,abstraction,ac,academic,acceplinge,accept,access,accomplishment,accuracy,accurate,achieved,ackerman,ackley,acknowledgements,acm,acomponent,acquainted,across,action,actions,active,activities,actly,actual,ad,add,added,adding,addition,additional,additionally,address,addressed,addressee,addresses,...,worked,working,works,workshop,workshops,world,worse,worst,would,write,writing,wrote,ws,wt,wu,wv,ww,wy,xn,xy,yamada,year,yearly,years,yet,yi,yis,yk,yn,yo,york,yorktown,young,youth,yr,yw,yy,zero,zo,zoom
0,0.000000,0.000000,0.000000,0.016256,0.000000,0.028044,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022135,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.022135,0.000000,0.000000,0.000000,0.000000,0.024383,0.032511,0.018696,0.037392,0.000000,0.022135,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.100539,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016256,0.000000,0.022135
1,0.000000,0.000000,0.000000,0.027156,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062466,0.036979,0.036979,0.000000,...,0.031233,0.000000,0.000000,0.00000,0.000000,0.031233,0.000000,0.000000,0.047988,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027156,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.028576,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024135,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028576,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(dense, dense)
print(np.matrix(cosine_sim))

[[1.         0.03005134 0.01296408 0.         0.         0.03133414
  0.03445396 0.03208275 0.44255733]
 [0.03005134 1.         0.00562574 0.         0.         0.03673798
  0.08287585 0.06649821 0.        ]
 [0.01296408 0.00562574 1.         0.         0.         0.02335712
  0.02277339 0.01615491 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.03133414 0.03673798 0.02335712 0.         0.         1.
  0.1927276  0.02965571 0.        ]
 [0.03445396 0.08287585 0.02277339 0.         0.         0.1927276
  1.         0.04533234 0.        ]
 [0.03208275 0.06649821 0.01615491 0.         0.         0.02965571
  0.04533234 1.         0.        ]
 [0.44255733 0.         0.         0.         0.         0.
  0.         0.         1.        ]]


In [ ]:
index = 0
for file in text:
  if index == 0:
    print(text[file])
    break
  else:
    index +=1

effect substrate concentration rate photosynthesis william chen aroshi ghosh muskaan grewal anoushka shah ryan sohn ap biology period due restrictions unable meet much might liked however still able work together zoom communicate text also took us attempts initially anticipated complete experiment coordinated efforts finish experiment earlier would time finish poster video future experiments would like include higher concentrations substrate g etc solidify theory limited number active sites certain point reaction rate stops increasing additionally experiment late fall sunlight bright would like attempt experiment summer sunlight brightest believe allow us accurate results less variability light source background information question variables data conclusion reflection photosynthesis one major processes plants converts water carbon dioxide light glucose oxygen occurs inside chloroplasts within plant cells order simulate photosynthesis controlled environment need living plant cells spin